In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 24.9 MB/s eta 0:00:00


In [ ]:
import os
import cv2
from PIL import Image
from glob import glob

# === Paths ===
BASE_PATH = '/content/drive/MyDrive/Colab Notebooks/Dataset/CAMO'
IMAGE_TRAIN_DIR = os.path.join(BASE_PATH, 'images/train')
IMAGE_TEST_DIR = os.path.join(BASE_PATH, 'images/test')
GT_MASK_DIR = os.path.join(BASE_PATH, 'GT')

LABEL_TRAIN_DIR = os.path.join(BASE_PATH, 'labels/train')
LABEL_TEST_DIR = os.path.join(BASE_PATH, 'labels/test')

# === Create label directories ===
os.makedirs(LABEL_TRAIN_DIR, exist_ok=True)
os.makedirs(LABEL_TEST_DIR, exist_ok=True)

# === Convert mask to YOLO polygon format ===
def mask_to_yolo(mask_path, image_shape):
    mask = cv2.imread(mask_path, 0)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    h, w = image_shape
    yolo_annotations = []

    for contour in contours:
        if len(contour) < 3:
            continue
        contour = contour.squeeze()
        points = []
        for x, y in contour:
            x_norm = x / w
            y_norm = y / h
            points.extend([x_norm, y_norm])
        label_line = f"0 " + " ".join([f"{p:.6f}" for p in points])
        yolo_annotations.append(label_line)
    return yolo_annotations

# === Main processing function ===
def process_split(image_dir, label_dir):
    image_files = glob(f"{image_dir}/*.jpg")
    for img_path in image_files:
        filename = os.path.basename(img_path).replace('.jpg', '')
        mask_path = os.path.join(GT_MASK_DIR, filename + '.png')  # assume GT masks are PNG
        if not os.path.exists(mask_path):
            print(f"[!] Missing mask: {mask_path}")
            continue

        with Image.open(img_path) as im:
            w, h = im.size
        yolo_labels = mask_to_yolo(mask_path, (h, w))

        label_path = os.path.join(label_dir, filename + '.txt')
        with open(label_path, 'w') as f:
            for line in yolo_labels:
                f.write(line + '\n')

# === Run for both train and test ===
process_split(IMAGE_TRAIN_DIR, LABEL_TRAIN_DIR)
process_split(IMAGE_TEST_DIR, LABEL_TEST_DIR)

print("✅ Conversion complete.")

✅ Conversion complete.


In [ ]:
yaml_content = """\
path: /content/drive/MyDrive/Colab Notebooks/Dataset/CAMO
train: images/train
val: images/test
test: images/test
nc: 1
names:
  - Camo
"""

with open('/content/drive/MyDrive/Colab Notebooks/Dataset/CAMO/data.yaml', 'w') as f:
    f.write(yaml_content)

print("✅ data.yaml created correctly.")

✅ data.yaml created correctly.


Convert GT Masks → YOLO Segmentation Format

In [ ]:
import os
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm

# Paths
GT_PATH = '/content/drive/MyDrive/Colab Notebooks/Dataset/CAMO/GT'
TRAIN_IMG_PATH = '/content/drive/MyDrive/Colab Notebooks/Dataset/CAMO/images/train'
TEST_IMG_PATH = '/content/drive/MyDrive/Colab Notebooks/Dataset/CAMO/images/test'
LABEL_PATH = '/content/drive/MyDrive/Colab Notebooks/Dataset/CAMO/labels'
os.makedirs(f'{LABEL_PATH}/Train', exist_ok=True)
os.makedirs(f'{LABEL_PATH}/Test', exist_ok=True)

# Get image filenames without extensions
train_files = [os.path.splitext(f)[0] for f in os.listdir(TRAIN_IMG_PATH) if f.endswith(('.jpg', '.png'))]
test_files = [os.path.splitext(f)[0] for f in os.listdir(TEST_IMG_PATH) if f.endswith(('.jpg', '.png'))]

def create_yolo_segmentation_labels(mask_path, save_path, class_id=0):
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    h, w = mask.shape

    _, thresh = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    label_lines = []
    for contour in contours:
        if len(contour) < 6:
            continue

        contour = contour.squeeze()
        norm_points = [(x / w, y / h) for x, y in contour]
        norm_points = [str(round(coord, 6)) for point in norm_points for coord in point]

        line = f"{class_id} " + " ".join(norm_points)
        label_lines.append(line)

    if label_lines:
        filename = os.path.splitext(os.path.basename(mask_path))[0] + ".txt"
        with open(os.path.join(save_path, filename), "w") as f:
            f.write("\n".join(label_lines))

# Process all masks from GT
all_masks = glob(f'{GT_PATH}/*.png')
print(f"Found {len(all_masks)} mask files in GT.")

for mask_file in tqdm(all_masks):
    base_name = os.path.splitext(os.path.basename(mask_file))[0]

    if base_name in train_files:
        dest_dir = os.path.join(LABEL_PATH, 'Train')
    elif base_name in test_files:
        dest_dir = os.path.join(LABEL_PATH, 'Test')
    else:
        print(f"⚠️ No matching image found for mask {base_name}")
        continue

    create_yolo_segmentation_labels(mask_file, dest_dir)


Found 1250 mask files in GT.


100%|██████████| 1250/1250 [06:13<00:00,  3.34it/s]


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 19.1 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Path to your YAML config
DATA_YAML = "/content/drive/MyDrive/Colab Notebooks/Dataset/CAMO/data.yaml"

# Create model instance (Nano segmentation model)
model = YOLO("yolov8s-seg.pt")  # yolov8s-seg.pt for better accuracy

# Start training
model.train(
    data=DATA_YAML,
    epochs=50,
    imgsz=416,
    batch=8,
    name="yolo_seg_CAMO",
    project="/content/drive/MyDrive/Colab Notebooks/Dataset/CAMO/runs"
)


Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cpu CPU (Intel Xeon 2.00GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Colab Notebooks/Dataset/CAMO/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo_seg_CAMO7, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective

100%|██████████| 755k/755k [00:00<00:00, 14.7MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

YOLOv8s-seg summary: 151 layers, 11,790,483 parameters, 11,790,467 gradients, 42.7 GFLOPs

Transferred 411/417 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
train: Fast image access ✅ (ping: 0.4±0.2 ms, read: 24.1±26.9 MB/s, size: 59.2 KB)


train: Scanning /content/drive/MyDrive/Colab Notebooks/Dataset/CAMO/labels/train.cache... 1000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1000/1000 [00:00<?, ?it/s]


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  12%|█▎        | 2/16 [00:02<00:19,  1.37s/it]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:26<00:00,  1.63s/it]


                   all        250        276      0.708      0.554      0.636      0.355      0.641      0.558      0.556      0.294
Speed: 0.4ms preprocess, 87.6ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to /content/drive/MyDrive/Colab Notebooks/Dataset/CAMO/runs/yolo_seg_CAMO7


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a3a30406650>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041, 

Evaluate the Model
To get metrics


In [ ]:
from ultralytics import YOLO
model = YOLO("path_to_best.pt")
metrics = model.val()
metrics = model.val()
print(metrics.box)   # For bounding box metrics
print(metrics.mask)  # For segmentation mask metrics


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_best.pt'

Visualize Metrics and Curves

In [ ]:
import matplotlib.pyplot as plt

# Visualizing one of the evaluation curves
metrics.plot_curve(curve='Precision-Recall(M)')  # Mask-based PR curve


Visualize Detection with Mask + Label

In [ ]:
import cv2
import numpy as np
from PIL import Image
from pathlib import Path

image_path = "/content/your_path/images/test/camourflage_01234.jpg"
mask_path = "/content/your_path/labels/test/camourflage_01234.txt"

image = cv2.imread(image_path)
h, w, _ = image.shape

with open(mask_path, 'r') as f:
    for line in f:
        cls, *points = map(float, line.strip().split())
        points = np.array(points).reshape(-1, 2) * [w, h]
        cv2.polylines(image, [points.astype(int)], isClosed=True, color=(0,255,0), thickness=2)
        cv2.putText(image, "Camo", tuple(points[0].astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)

plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()


Test Code for displaying single image

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
import os

# Choose a file to visualize (you can change this)
img_name = 'camourflage_00001.jpg'  # make sure this exists
image_path = f'/content/drive/MyDrive/Colab Notebooks/Dataset/CAMO/images/train/{img_name}'
label_path = f'/content/drive/MyDrive/Colab Notebooks/Dataset/CAMO/labels/Train/{img_name.replace(".jpg", ".txt")}'

# Load image
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
h, w, _ = image.shape

# Draw polygons from label file
with open(label_path, 'r') as f:
    lines = f.readlines()

for line in lines:
    parts = list(map(float, line.strip().split()))
    class_id = int(parts[0])
    coords = np.array(parts[1:]).reshape(-1, 2)  # [x, y] pairs
    coords *= [w, h]  # convert to pixel coordinates
    coords = coords.astype(int)

    cv2.polylines(image, [coords], isClosed=True, color=(255, 0, 0), thickness=2)

# Show the image
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.title(f'Segmentation mask: {img_name}')
plt.axis('off')
plt.show()
